In [1]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq
import pandasql as ps
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from joblib import dump,load
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

credentials_path = r"service_account_json_file_path"
credentials = service_account.Credentials.from_service_account_file(credentials_path)
project_id = "bq_project_id"

sql = """

SELECT agg.*,val.* EXCEPT (user_pseudo_id), ltv90.itemRevenue90 FROM

(select * from 
(SELECT 
user_pseudo_id,
traffic_source.medium,
traffic_source.name,
traffic_source.source,
device.mobile_brand_name,
geo.country,
device.category,
row_number() OVER (PARTITION BY user_pseudo_id order by event_timestamp asc) as cnt
FROM 
--- Update the below dataset to match your GA4 dataset and project
`bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*` LEFT JOIN UNNEST(items) AS items
WHERE _table_suffix >= '20201101' AND _table_suffix <= '20210131'
AND DATE(TIMESTAMP_MICROS(user_first_touch_timestamp)) BETWEEN '2020-11-01' AND '2020-11-30'
AND DATE_DIFF(DATE(TIMESTAMP_MICROS(event_timestamp)) , DATE(TIMESTAMP_MICROS(user_first_touch_timestamp)) , DAY) <=15)
WHERE cnt = 1) as agg

LEFT JOIN

(SELECT 
date(timestamp_micros(user_first_touch_timestamp)) as fdate,
user_pseudo_id,
--device.category,
--device.mobile_brand_name,
--geo.country,
--geo.city,
--geo.region,
--traffic_source.medium,
--traffic_source.name,
--traffic_source.source,
count(distinct (select value.int_value from unnest (event_params) WHERE KEY = 'ga_session_id')) as sessionCnt,
count(distinct case when event_name = 'session_start' THEN date(timestamp_micros(event_timestamp))END) as sessionDate,
count(distinct item_brand) itemBrandCount,
count(distinct item_category) itemCatCount,
count(distinct case when event_name = 'view_promotion' THEN event_timestamp END) as viwePromotion,
count(distinct case when event_name = 'select_promotion' THEN event_timestamp END) as SelectPromotion,


COUNT(DISTINCT case when event_name = 'view_item' THEN item_name END) AS itemViewCnt,
COUNT(DISTINCT case when event_name = 'select_item' THEN item_name END) AS itemSelectCnt,

count(distinct case when event_name = 'add_payment_info' THEN event_timestamp END) as paymetInfoAdd,
count(distinct case when event_name = 'add_shipping_info' THEN event_timestamp END) as shippingInfoAdd,


count(distinct CASE WHEN  event_name = 'scroll' THEN (select value.string_value from unnest (event_params) WHERE KEY = 'page_location') END) ScrollpageLocationCnt,
count(distinct CASE WHEN  event_name = 'scroll' THEN (select value.string_value from unnest (event_params) WHERE KEY = 'page_title') END) ScrollpageTitleCnt,

count(distinct CASE WHEN  event_name = 'page_view' THEN (select value.string_value from unnest (event_params) WHERE KEY = 'page_location') END) pageViewPageLocationCnt,
count(distinct CASE WHEN  event_name = 'page_view' THEN (select value.string_value from unnest (event_params) WHERE KEY = 'page_title') END) pageViewPageTitleCnt,

--Item views
COUNT(distinct CASE WHEN event_name = 'view_item' THEN event_timestamp ELSE NULL END) AS itemViews,
--Add-to-carts
COUNT( DISTINCT CASE WHEN event_name = 'add_to_cart' THEN event_timestamp ELSE NULL END) AS addToCarts,
COUNT( DISTINCT CASE WHEN event_name = 'add_to_cart' THEN items.item_id ELSE NULL END) AS addToItemId,

COUNT( DISTINCT CASE WHEN event_name = 'view_search_results' THEN event_timestamp ELSE NULL END) AS searchResultViewedCnt,
count(distinct case when event_name = 'begin_checkout' THEN event_timestamp else null end) as checkOut,
--Ecommerce Purchases
COUNT(CASE WHEN event_name = 'purchase' THEN ecommerce.transaction_id ELSE NULL END) AS ecommercePurchases, 
--Purchase-to-view-rate
(CASE WHEN COUNT(CASE WHEN event_name = 'view_item' THEN  event_timestamp ELSE NULL END) = 0 THEN 0
ELSE COUNT(DISTINCT CASE WHEN event_name = 'purchase' THEN event_timestamp  ELSE NULL END) /
COUNT(DISTINCT CASE WHEN event_name = 'view_item' THEN event_timestamp  ELSE NULL END) END  * 100) AS purchaseToViewRate,
--Item purchase quantity
COUNT(distinct CASE WHEN event_name = 'purchase' THEN items.item_name  ELSE NULL END) AS itemPurchaseName,

IFNULL(SUM(CASE WHEN event_name = 'purchase' THEN items.quantity  ELSE NULL END),0) AS itemPurchaseQuantity,
--Item revenue
IFNULL(SUM(item_revenue),0) AS itemRevenue15,
IFNULL(SUM(CASE WHEN event_name = 'purchase' THEN item_revenue ELSE NULL END),0) AS itemRevenue2

FROM 
--- Update the below dataset to match your GA4 dataset and project
`bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*` LEFT JOIN UNNEST(items) AS items
WHERE _table_suffix >= '20201101' AND _table_suffix <= '20210131'
AND DATE(TIMESTAMP_MICROS(user_first_touch_timestamp)) BETWEEN '2020-11-01' AND '2020-11-30'
AND DATE_DIFF(DATE(TIMESTAMP_MICROS(event_timestamp)) , DATE(TIMESTAMP_MICROS(user_first_touch_timestamp)) , DAY) <=15
GROUP BY 1,2
) as val on agg.user_pseudo_id = val.user_pseudo_id

LEFT JOIN

(SELECT 
date(timestamp_micros(user_first_touch_timestamp)) as fdate,
user_pseudo_id,

IFNULL(SUM(item_revenue),0) AS itemRevenue90,

FROM 
--- Update the below dataset to match your GA4 dataset and project
`bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*` LEFT JOIN UNNEST(items) AS items
WHERE _table_suffix >= '20201101' AND _table_suffix <= '20210131'
AND DATE(TIMESTAMP_MICROS(user_first_touch_timestamp)) BETWEEN '2020-11-01' AND '2020-11-30'
AND DATE_DIFF(DATE(TIMESTAMP_MICROS(event_timestamp)) , DATE(TIMESTAMP_MICROS(user_first_touch_timestamp)) , DAY) <=90
GROUP BY 1,2
) as ltv90 on agg.user_pseudo_id = ltv90.user_pseudo_id


"""

In [5]:
##df = pandas_gbq.read_gbq(sql, project_id= project_id)
df = pd.read_pickle("df.pkl")

Downloading: 100%|██████████|


In [6]:
gdp = pd.read_csv(r"C:\Users\w11\Desktop\Project3\imf-dm-export.csv")
gdp

,"GDP per capita, current prices\n (U.S. dollars per capita)",1980,1981,1982,1983,1984,1985,1986,1987,1988,...,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029
0,Afghanistan,no data,no data,no data,no data,no data,no data,no data,no data,no data,...,611.268,423.993,422.229,no data,no data,no data,no data,no data,no data,no data
1,Albania,728.359,817.734,824.542,815.529,788.332,788.801,855.724,832.245,805.046,...,5278.986,6259.762,6657.637,7956.559,8924.317,9474.065,10135.055,10827.923,11578.584,12389.74
2,Algeria,2462.085,2502.13,2446.588,2514.256,2640.191,2988.545,2929.092,2935.816,2326.571,...,3757.585,4169.984,4981.822,5323.635,5721.678,5869.435,5998.622,6073.858,6129.969,6170.756
3,Andorra,no data,no data,no data,no data,no data,no data,no data,no data,no data,...,36973.845,41806.876,41378.881,43784.571,44899.596,45570.14,46093.258,46597.511,47098.423,47563.642
4,Angola,802.627,731.427,712.576,723.654,747.325,817.068,743.735,828.958,875.526,...,1709.283,2169.648,3439.391,2565.912,2431.58,2444.365,2455.711,2528.702,2571.26,2617.815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,Latin America and the Caribbean,2536.298,2684.736,2147.602,1908.459,1947.855,2052.065,1986.52,2075.082,2300.953,...,7043.843,8140.639,9241.181,10291.151,10878.689,11242.058,11759.286,12286.797,12788.801,13301.689
223,Major advanced economies (G7),11275.803,11554.398,11436.111,12005.038,12568.159,13252.936,15946.884,18227.138,20367.02,...,50839.174,55977.766,57110.146,60221.173,62412.739,64658.942,67067.991,69485.17,72050.992,74590.436
224,Middle East and Central Asia,2108.544,2179.306,2101.667,2075.924,2039.69,2041.836,2023.437,2329.576,2385.981,...,4090.533,4721.043,5542.042,5582.4,5744.87,5887.207,6124.102,6384.882,6637.653,6908.869
225,Other advanced economies,6888.89,6933.195,6795.563,6702.399,6951.035,6864.222,8294.847,10109.366,11791.237,...,42672.028,49582.899,49454.631,49696.398,51506.626,53692.239,55812.83,57915.546,60155.29,62416.984


In [7]:
gdp = gdp[['GDP per capita, current prices\n (U.S. dollars per capita)','2020','2021']]
gdp = gdp.rename(columns= {'GDP per capita, current prices\n (U.S. dollars per capita)':'country'})
gdp = gdp.rename(columns= {'2020':'gdp_2020_value'})
gdp = gdp.rename(columns= {'2021':'gdp_2021_value'})

In [8]:
merged_df = pd.merge(df, gdp, on='country',  how='left')
merged_df

,user_pseudo_id,medium,name,source,mobile_brand_name,country,category,cnt,fdate,sessionCnt,...,checkOut,ecommercePurchases,purchaseToViewRate,itemPurchaseName,itemPurchaseQuantity,itemRevenue15,itemRevenue2,itemRevenue90,gdp_2020_value,gdp_2021_value
0,10076354.1924185783,organic,(organic),google,Google,United States,desktop,1,2020-11-29,1,...,0,0,0.0,0,0,0.0,0.0,0.0,64367.435,70995.794
1,1011144.1893306037,organic,(organic),google,Apple,United States,desktop,1,2020-11-21,1,...,0,0,0.0,0,0,0.0,0.0,0.0,64367.435,70995.794
2,10674454.7042425090,referral,(referral),<Other>,Samsung,United Kingdom,mobile,1,2020-11-16,1,...,0,0,0.0,0,0,0.0,0.0,0.0,40245.884,46704.465
3,10854466.6205054217,organic,(organic),google,<Other>,China,mobile,1,2020-11-03,1,...,0,0,0.0,0,0,0.0,0.0,0.0,10525.001,12572.071
4,11606374.3149985852,organic,(organic),google,<Other>,Greece,mobile,1,2020-11-07,1,...,0,0,0.0,0,0,0.0,0.0,0.0,17570.395,20115.924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71733,9347237.4321514307,<Other>,<Other>,<Other>,Apple,India,mobile,1,2020-11-24,1,...,0,0,0.0,0,0,0.0,0.0,0.0,1915.552,2250.179
71734,9416281.3655045282,organic,(organic),<Other>,Mozilla,United States,desktop,1,2020-11-29,1,...,0,0,0.0,0,0,0.0,0.0,0.0,64367.435,70995.794
71735,96581384.5015113025,(none),(direct),(direct),Google,United States,desktop,1,2020-11-12,1,...,0,0,0.0,0,0,0.0,0.0,0.0,64367.435,70995.794
71736,9738471.8732397597,referral,(referral),<Other>,Apple,France,mobile,1,2020-11-21,1,...,0,0,0.0,0,0,0.0,0.0,0.0,40529.093,45161.226


In [9]:
merged_df['gdp_2020_value'] = merged_df['gdp_2020_value'].astype(float)
merged_df['gdp_2021_value'] = merged_df['gdp_2021_value'].astype(float)
merged_df['Avg_gdp'] =  merged_df[['gdp_2020_value','gdp_2021_value']].mean(axis = 1)

In [10]:
country_filtered = merged_df[merged_df['country'].isin(["United States", "Canada", "Germany", "India", "France", "Taiwan",'Italy', "Japan","Spain", "China", "Singapore", "South Korea", "Netherlands", 'Turkey'])]

In [11]:
country_filtered.loc[:, 'itemRevenue90'] = country_filtered['itemRevenue90'].apply(lambda x: x if x > 0 else 1e-10)
country_filtered.loc[:, 'LogLtv90'] = np.log(country_filtered['itemRevenue90'])
country_filtered.loc[:, 'LogGDP'] = np.log(country_filtered['Avg_gdp'])

C:\Users\w11\AppData\Local\Temp\ipykernel_18592\1260032352.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_filtered.loc[:, 'LogLtv90'] = np.log(country_filtered['itemRevenue90'])
C:\Users\w11\AppData\Local\Temp\ipykernel_18592\1260032352.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_filtered.loc[:, 'LogGDP'] = np.log(country_filtered['Avg_gdp'])


In [12]:
dummies_df = pd.get_dummies(country_filtered[['medium', 'mobile_brand_name', 'country', 'category']], drop_first=True, dtype=int)
dummy_df = pd.concat([country_filtered,dummies_df], axis = 1)


In [13]:
x = dummy_df.drop(['itemRevenue2','itemRevenue90', 'LogLtv90', 'user_pseudo_id','medium','name','source','mobile_brand_name','country','category','cnt','fdate'], axis = 1)
y = (dummy_df.LogLtv90)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [21]:
pd.set_option('display.max_columns', None)
X_train

,sessionCnt,sessionDate,itemBrandCount,itemCatCount,viwePromotion,SelectPromotion,itemViewCnt,itemSelectCnt,paymetInfoAdd,shippingInfoAdd,ScrollpageLocationCnt,ScrollpageTitleCnt,pageViewPageLocationCnt,pageViewPageTitleCnt,itemViews,addToCarts,addToItemId,searchResultViewedCnt,checkOut,ecommercePurchases,purchaseToViewRate,itemPurchaseName,itemPurchaseQuantity,itemRevenue15,gdp_2020_value,gdp_2021_value,Avg_gdp,LogGDP,medium_(none),medium_<Other>,medium_cpc,medium_organic,medium_referral,mobile_brand_name_Apple,mobile_brand_name_Google,mobile_brand_name_Huawei,mobile_brand_name_Microsoft,mobile_brand_name_Mozilla,mobile_brand_name_Samsung,mobile_brand_name_Xiaomi,country_China,country_France,country_Germany,country_India,country_Italy,country_Japan,country_Netherlands,country_Singapore,country_South Korea,country_Spain,country_Taiwan,country_Turkey,country_United States,category_mobile,category_tablet
41231,1,1,2,5,1,0,21,0,0,0,4,4,6,6,4,0,0,0,0,0,0.0,0,0,0.0,31728.306,35125.524,33426.9150,10.417117,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
11197,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0.0,0,0,0.0,64367.435,70995.794,67681.6145,11.122570,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
53558,1,1,2,2,0,0,13,11,0,0,1,1,6,6,7,4,13,0,0,0,0.0,0,0,0.0,64367.435,70995.794,67681.6145,11.122570,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
49210,1,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0.0,0,0,0.0,31788.997,36401.883,34095.4400,10.436919,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
62877,1,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0.0,0,0,0.0,43573.195,52520.512,48046.8535,10.779932,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57960,1,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0.0,0,0,0.0,64367.435,70995.794,67681.6145,11.122570,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
70415,2,1,2,2,0,0,11,0,0,0,2,2,3,3,3,0,0,2,0,0,0.0,0,0,0.0,46711.984,51460.987,49086.4855,10.801339,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
49482,1,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0.0,0,0,0.0,64367.435,70995.794,67681.6145,11.122570,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1120,1,1,1,2,1,0,12,0,0,2,3,3,7,6,4,0,0,0,4,0,0.0,0,0,0.0,40171.929,40114.287,40143.1080,10.600206,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0


In [16]:
X_test

,sessionCnt,sessionDate,itemBrandCount,itemCatCount,viwePromotion,SelectPromotion,itemViewCnt,itemSelectCnt,paymetInfoAdd,shippingInfoAdd,...,country_Japan,country_Netherlands,country_Singapore,country_South Korea,country_Spain,country_Taiwan,country_Turkey,country_United States,category_mobile,category_tablet
54188,1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
64000,2,2,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
33663,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
360,4,3,1,1,4,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
44558,1,1,1,1,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52656,1,1,1,1,3,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
29324,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
57014,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
53013,1,1,1,1,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
y_train

41231   -23.025851
11197   -23.025851
53558   -23.025851
49210   -23.025851
62877   -23.025851
           ...    
57960   -23.025851
70415   -23.025851
49482   -23.025851
1120    -23.025851
20593   -23.025851
Name: LogLtv90, Length: 44290, dtype: float64

In [18]:
y_test

54188   -23.025851
64000   -23.025851
33663   -23.025851
360     -23.025851
44558   -23.025851
           ...    
52656   -23.025851
29324   -23.025851
57014   -23.025851
53013   -23.025851
5705    -23.025851
Name: LogLtv90, Length: 11073, dtype: float64

In [19]:
# Modeli başlatma ve eğitme
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)  # 100 ağaç kullanıyoruz
rf_model.fit(X_train, y_train)

# Test seti üzerinde tahmin yapma
y_pred_rf = rf_model.predict(X_test)

# Performans metriklerini hesaplama
test_score_rf = r2_score(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)

print(f"Test R2 Score: {test_score_rf:.3f}")
print(f"Mean Squared Error: {mse_rf:.3f}")
print(f"Root Mean Squared Error: {rmse_rf:.3f}")

Test R2 Score: 0.816
Mean Squared Error: 2.273
Root Mean Squared Error: 1.508


In [20]:
dump(rf_model, 'randomForest100Model_new.pkl')

['randomForest100Model_new.pkl']